# Вебинар 5. Ранжирование и Гибридные рекоммендательные системы

In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# from src.metrics import precision_at_k, recall_at_k
from utils import prefilter_items

In [66]:
data = pd.read_csv('../webinar_2/webinar_2/data/retail_train.csv')
item_features = pd.read_csv('../webinar_2/webinar_2/data/product.csv')
user_features = pd.read_csv('../webinar_2/webinar_2/data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [67]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [68]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [69]:
user_features['age_desc'].unique()

array(['65+', '45-54', '25-34', '35-44', '19-24', '55-64'], dtype=object)

In [70]:
user_features['marital_status_code'].unique()

array(['A', 'U', 'B'], dtype=object)

In [71]:
user_features['household_size_desc'].unique()

array(['2', '3', '4', '1', '5+'], dtype=object)

## 1. Filter items

In [72]:
data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [73]:
n_items_before = data_train['item_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)

n_items_after = data_train_filtered['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


# 2. Prepare data set

## 2.1 Prepare csr train matrix

In [74]:
#train
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,25671,26081,26093,26190,26355,26426,26540,26601,26636,26691,...,17328742,17329473,17329749,17330255,17330511,17381856,17382205,17383227,17827644,17829232
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2.2 Prepare CSR test matrix

In [75]:
#test
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]
test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit


In [76]:
# переведем в формат sparse matrix
sparse_test_user_item = csr_matrix(test_user_item_matrix).tocsr()

In [77]:
#train
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

## 3. Prepare user and item features

In [78]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
user_feat.shape

(2499, 7)

In [80]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [81]:
item_feat.shape

(86865, 6)

## Encoding features

In [82]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [83]:
user_feat_lightfm.head(2)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Init model

In [84]:
model = LightFM(no_components=30,
#                 loss='bpr',
                loss='warp',
                learning_rate=0.05, 
                item_alpha=0.1,
                user_alpha=0.1, 
                random_state=42)

## Train

In [85]:
model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=15, 
          num_threads=4,
          verbose=False) 

In [86]:
user_emb = model.get_user_representations(features=csr_matrix(user_feat_lightfm.values).tocsr())

In [87]:
user_emb[0]  # biases

array([-94.44411,   0.     ,   0.     , ..., -98.41175, -67.35313,
         0.     ], dtype=float32)

In [88]:
user_emb[0].shape

(2499,)

In [89]:
user_emb[1]  # embeddings

array([[-1.8431123e-10, -1.6083463e-09, -2.7885445e-09, ...,
        -1.7285773e-10,  3.0333247e-09, -3.9324544e-11],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [-4.4856061e-09, -2.4581723e-09,  5.7605964e-10, ...,
        -1.7135227e-10,  2.1238833e-09, -1.4274275e-09],
       [-3.3761289e-09, -1.4854289e-09,  1.2941355e-09, ...,
        -1.9334563e-09,  5.1197713e-10, -1.0599461e-09],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)

In [90]:
user_emb[1].shape

(2499, 30)

In [91]:
item_emb = model.get_item_representations(features=csr_matrix(item_feat_lightfm.values).tocsr())

In [92]:
item_emb[0].shape

(86865,)

In [93]:
item_emb[1].shape

(86865, 30)

In [94]:
train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

train_precision

0.051700678

In [95]:
test_precision = precision_at_k(model, test_interactions = sparse_test_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

test_precision

0.0022537974

In [34]:
test_item_ids = np.array([1, 2, 3, 200, 1200])

predictions = model.predict(user_ids=0, item_ids=test_item_ids,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=4)
predictions

array([-94.74061, -94.18075, -94.21038, -94.52779, -94.16204],
      dtype=float32)

In [96]:
model = LightFM(no_components=20,
#                 loss='bpr',
                loss='warp', learning_rate=0.1, item_alpha=0.1, user_alpha=0.1, random_state=42)
model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=15, 
          num_threads=4,
          verbose=False) 

In [97]:
train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

train_precision

0.20064026

In [98]:
test_precision = precision_at_k(model, test_interactions = sparse_test_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

test_precision

0.0028417443

In [103]:
no_comp = [20, 30, 40]
losses = ['bpr']
lr = [ 0.05, 0.1, 0.2, 0.3]
it_a = [ 0.1, 0.2, 0.3]
us_a = [ 0.1, 0.2, 0.3]

In [104]:
for elem in no_comp:
    for elem1 in losses:
        for elem2 in lr:
            for elem3 in it_a:
                for elem4 in us_a:
                    model = LightFM(no_components=elem,loss=elem1, learning_rate=elem2, item_alpha=elem3, user_alpha=elem4, random_state=42)
                    model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
                      sample_weight=coo_matrix(user_item_matrix),
                      user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                      item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                      epochs=15, num_threads=4, verbose=False) 
                    train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()
                    test_precision = precision_at_k(model, test_interactions = sparse_test_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

                    print(f'no_comp: {elem}, loss: {elem1}, learn_r: {elem2}, item_alpha {elem3}, user_alpha {elem4}')
                    print(f'train_precision: {train_precision}, test_precision: {test_precision}')

no_comp: 20, loss: bpr, learn_r: 0.05, item_alpha 0.1, user_alpha 0.1
train_precision: 0.0, test_precision: 0.0011758941691368818
no_comp: 20, loss: bpr, learn_r: 0.05, item_alpha 0.1, user_alpha 0.2
train_precision: 0.0, test_precision: 0.0006859382847324014
no_comp: 20, loss: bpr, learn_r: 0.05, item_alpha 0.1, user_alpha 0.3
train_precision: 0.0018407362513244152, test_precision: 0.0013718765694648027
no_comp: 20, loss: bpr, learn_r: 0.05, item_alpha 0.2, user_alpha 0.1
train_precision: 0.006642656400799751, test_precision: 0.0007839294848963618
no_comp: 20, loss: bpr, learn_r: 0.05, item_alpha 0.2, user_alpha 0.2
train_precision: 0.00208083214238286, test_precision: 0.000979911768808961
no_comp: 20, loss: bpr, learn_r: 0.05, item_alpha 0.2, user_alpha 0.3
train_precision: 0.0, test_precision: 0.0006859382847324014
no_comp: 20, loss: bpr, learn_r: 0.05, item_alpha 0.3, user_alpha 0.1
train_precision: 0.00104041607119143, test_precision: 0.0008819206850603223
no_comp: 20, loss: bpr, 

no_comp: 30, loss: bpr, learn_r: 0.2, item_alpha 0.2, user_alpha 0.2
train_precision: 0.0012004801537841558, test_precision: 0.0016658501699566841
no_comp: 30, loss: bpr, learn_r: 0.2, item_alpha 0.2, user_alpha 0.3
train_precision: 0.005282113328576088, test_precision: 0.0008819206850603223
no_comp: 30, loss: bpr, learn_r: 0.2, item_alpha 0.3, user_alpha 0.1
train_precision: 0.037695080041885376, test_precision: 0.0010779029689729214
no_comp: 30, loss: bpr, learn_r: 0.2, item_alpha 0.3, user_alpha 0.2
train_precision: 0.0012004801537841558, test_precision: 0.002253797370940447
no_comp: 30, loss: bpr, learn_r: 0.2, item_alpha 0.3, user_alpha 0.3
train_precision: 0.00016006402438506484, test_precision: 0.0008819206850603223
no_comp: 30, loss: bpr, learn_r: 0.3, item_alpha 0.1, user_alpha 0.1
train_precision: 0.001280512078665197, test_precision: 0.0008819206850603223
no_comp: 30, loss: bpr, learn_r: 0.3, item_alpha 0.1, user_alpha 0.2
train_precision: 0.0012004801537841558, test_precisi

ValueError: Not all estimated parameters are finite, your model may have diverged. Try decreasing the learning rate or normalising feature values and sample weights

In [105]:
no_comp = [30, 40]
losses = ['warp']
lr = [ 0.05, 0.1, 0.2, 0.3]
it_a = [ 0.1, 0.2, 0.3]
us_a = [ 0.1, 0.2, 0.3]

In [106]:
for elem in no_comp:
    for elem1 in losses:
        for elem2 in lr:
            for elem3 in it_a:
                for elem4 in us_a:
                    model = LightFM(no_components=elem,loss=elem1, learning_rate=elem2, item_alpha=elem3, user_alpha=elem4, random_state=42)
                    model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
                      sample_weight=coo_matrix(user_item_matrix),
                      user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                      item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                      epochs=15, num_threads=4, verbose=False) 
                    train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()
                    test_precision = precision_at_k(model, test_interactions = sparse_test_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

                    print(f'no_comp: {elem}, loss: {elem1}, learn_r: {elem2}, item_alpha {elem3}, user_alpha {elem4}')
                    print(f'train_precision: {train_precision}, test_precision: {test_precision}')

no_comp: 30, loss: warp, learn_r: 0.05, item_alpha 0.1, user_alpha 0.1
train_precision: 0.05170067772269249, test_precision: 0.002253797370940447
no_comp: 30, loss: warp, learn_r: 0.05, item_alpha 0.1, user_alpha 0.2
train_precision: 0.17262905836105347, test_precision: 0.005389514844864607
no_comp: 30, loss: warp, learn_r: 0.05, item_alpha 0.1, user_alpha 0.3
train_precision: 0.19023610651493073, test_precision: 0.00597746204584837
no_comp: 30, loss: warp, learn_r: 0.05, item_alpha 0.2, user_alpha 0.1
train_precision: 0.0, test_precision: 0.0027437531389296055
no_comp: 30, loss: warp, learn_r: 0.05, item_alpha 0.2, user_alpha 0.2
train_precision: 0.17999200522899628, test_precision: 0.005193532910197973
no_comp: 30, loss: warp, learn_r: 0.05, item_alpha 0.2, user_alpha 0.3
train_precision: 0.0, test_precision: 0.0018618324538692832
no_comp: 30, loss: warp, learn_r: 0.05, item_alpha 0.3, user_alpha 0.1
train_precision: 0.15174070000648499, test_precision: 0.0012738852528855205
no_comp:

no_comp: 40, loss: warp, learn_r: 0.2, item_alpha 0.2, user_alpha 0.3
train_precision: 0.0, test_precision: 0.0011758941691368818
no_comp: 40, loss: warp, learn_r: 0.2, item_alpha 0.3, user_alpha 0.1
train_precision: 0.0, test_precision: 0.001959823537617922
no_comp: 40, loss: warp, learn_r: 0.2, item_alpha 0.3, user_alpha 0.2
train_precision: 0.0, test_precision: 0.0014698677696287632
no_comp: 40, loss: warp, learn_r: 0.2, item_alpha 0.3, user_alpha 0.3
train_precision: 0.09771909564733505, test_precision: 0.0
no_comp: 40, loss: warp, learn_r: 0.3, item_alpha 0.1, user_alpha 0.1
train_precision: 0.1663065403699875, test_precision: 0.0008819206268526614
no_comp: 40, loss: warp, learn_r: 0.3, item_alpha 0.1, user_alpha 0.2
train_precision: 0.00416166428476572, test_precision: 0.004017638508230448
no_comp: 40, loss: warp, learn_r: 0.3, item_alpha 0.1, user_alpha 0.3
train_precision: 0.0, test_precision: 0.0010779029689729214
no_comp: 40, loss: warp, learn_r: 0.3, item_alpha 0.2, user_alp

Лушие результаты 
BPR
no_comp: 20, loss: bpr, learn_r: 0.1, item_alpha 0.3, user_alpha 0.1
train_precision: 0.03241296485066414, test_precision: 0.003821656107902527

no_comp: 40, loss: bpr, learn_r: 0.3, item_alpha 0.2, user_alpha 0.1
train_precision: 0.004001600667834282, test_precision: 0.0046055857092142105

WARP

no_comp: 30, loss: warp, learn_r: 0.05, item_alpha 0.1, user_alpha 0.3
train_precision: 0.19023610651493073, test_precision: 0.00597746204584837

no_comp: 40, loss: warp, learn_r: 0.1, item_alpha 0.2, user_alpha 0.1
train_precision: 0.23609444499015808, test_precision: 0.005487506277859211

no_comp: 20, loss: warp, learn_r: 0.05, item_alpha 0.1, user_alpha 0.3
train_precision: 0.19023610651493073, test_precision: 0.00597746204584837

В целом, результаты при двух разных loss не очень хорошие, но при WARP
Я сделала небольшой подбор параметров, но все очень долго считается
Поэтому возможно при большей сетке параметров найдутся параметры, при которых результаты будут лучше 



Домашнее задание по желанию:
Из нотбука вебинара
Для рекомендательной модели при помощи библиотеки ligthFM:
1) Посчитать метрики на тестовом датасете
2) Попытаться улучшить качество подбором параметров
3) Обучить на BPR лоссе и сравнить по качеству
4) Сделать выводы

еще результаты запусков

no_comp: 20, loss: warp, learn_r: 0.05, item_alpha 0.1, user_alpha 0.1
train_precision: 0.05170067772269249, test_precision: 0.002253797370940447

no_comp: 20, loss: warp, learn_r: 0.05, item_alpha 0.1, user_alpha 0.2
train_precision: 0.19023610651493073, test_precision: 0.005389514844864607

no_comp: 20, loss: warp, learn_r: 0.05, item_alpha 0.1, user_alpha 0.3
train_precision: 0.19023610651493073, test_precision: 0.00597746204584837

no_comp: 20, loss: warp, learn_r: 0.05, item_alpha 0.2, user_alpha 0.1
train_precision: 0.049139656126499176, test_precision: 0.002841744339093566



no_comp: 10, loss: bpr, learn_r: 0, item_alpha 0, user_alpha 0
train_precision: 0.0006402560975402594, test_precision: 0.0005879470845684409
no_comp: 10, loss: bpr, learn_r: 0, item_alpha 0, user_alpha 0.1
train_precision: 0.0006402560975402594, test_precision: 0.0005879470845684409
no_comp: 10, loss: bpr, learn_r: 0, item_alpha 0, user_alpha 0.5
train_precision: 0.0006402560975402594, test_precision: 0.0005879470845684409
no_comp: 10, loss: bpr, learn_r: 0, item_alpha 0, user_alpha 1
train_precision: 0.0006402560975402594, test_precision: 0.0005879470845684409
no_comp: 10, loss: bpr, learn_r: 0, item_alpha 0.1, user_alpha 0
train_precision: 0.0006402560975402594, test_precision: 0.0005879470845684409
no_comp: 10, loss: bpr, learn_r: 0, item_alpha 0.1, user_alpha 0.1
train_precision: 0.0006402560975402594, test_precision: 0.0005879470845684409
no_comp: 10, loss: bpr, learn_r: 0, item_alpha 0.1, user_alpha 0.5
train_precision: 0.0006402560975402594, test_precision: 0.0005879470845684409
no_comp: 10, loss: bpr, learn_r: 0, item_alpha 0.1, user_alpha 1
train_precision: 0.0006402560975402594, test_precision: 0.0005879470845684409
no_comp: 10, loss: bpr, learn_r: 0, item_alpha 0.5, user_alpha 0
train_precision: 0.0006402560975402594, test_precision: 0.0005879470845684409
no_comp: 10, loss: bpr, learn_r: 0, item_alpha 0.5, user_alpha 0.1
train_precision: 0.0006402560975402594, test_precision: 0.0005879470845684409
no_comp: 10, loss: bpr, learn_r: 0, item_alpha 0.5, user_alpha 0.5
train_precision: 0.0006402560975402594, test_precision: 0.0005879470845684409
no_comp: 10, loss: bpr, learn_r: 0, item_alpha 0.5, user_alpha 1
train_precision: 0.0006402560975402594, test_precision: 0.0005879470845684409
no_comp: 10, loss: bpr, learn_r: 0, item_alpha 1, user_alpha 0
train_precision: 0.0006402560975402594, test_precision: 0.0005879470845684409
no_comp: 10, loss: bpr, learn_r: 0, item_alpha 1, user_alpha 0.1
train_precision: 0.0006402560975402594, test_precision: 0.0005879470845684409
no_comp: 10, loss: bpr, learn_r: 0, item_alpha 1, user_alpha 0.5
train_precision: 0.0006402560975402594, test_precision: 0.0005879470845684409
no_comp: 10, loss: bpr, learn_r: 0, item_alpha 1, user_alpha 1
train_precision: 0.0006402560975402594, test_precision: 0.0005879470845684409
no_comp: 10, loss: bpr, learn_r: 0.05, item_alpha 0, user_alpha 0
train_precision: 0.0006402560975402594, test_precision: 0.002841744339093566
no_comp: 10, loss: bpr, learn_r: 0.05, item_alpha 0, user_alpha 0.1
train_precision: 0.0012805121950805187, test_precision: 0.0013718765694648027
no_comp: 10, loss: bpr, learn_r: 0.05, item_alpha 0, user_alpha 0.5
train_precision: 0.0007202880806289613, test_precision: 0.00029397354228422046
no_comp: 10, loss: bpr, learn_r: 0.05, item_alpha 0, user_alpha 1
train_precision: 0.00016006402438506484, test_precision: 0.0010779029689729214
no_comp: 10, loss: bpr, learn_r: 0.05, item_alpha 0.1, user_alpha 0
train_precision: 0.00016006402438506484, test_precision: 0.0008819206850603223
no_comp: 10, loss: bpr, learn_r: 0.05, item_alpha 0.1, user_alpha 0.1
train_precision: 0.0, test_precision: 0.0010779029689729214
no_comp: 10, loss: bpr, learn_r: 0.05, item_alpha 0.1, user_alpha 0.5
train_precision: 0.0004001600609626621, test_precision: 0.0012738853693008423
no_comp: 10, loss: bpr, learn_r: 0.05, item_alpha 0.1, user_alpha 1
train_precision: 0.0, test_precision: 0.0
no_comp: 10, loss: bpr, learn_r: 0.05, item_alpha 0.5, user_alpha 0
train_precision: 0.0002400960511295125, test_precision: 0.002449779538437724
no_comp: 10, loss: bpr, learn_r: 0.05, item_alpha 0.5, user_alpha 0.1
train_precision: 0.0002400960511295125, test_precision: 0.0
no_comp: 10, loss: bpr, learn_r: 0.05, item_alpha 0.5, user_alpha 0.5
train_precision: 0.00096038420451805, test_precision: 0.0003919647424481809
no_comp: 10, loss: bpr, learn_r: 0.05, item_alpha 0.5, user_alpha 1
train_precision: 0.0007202881388366222, test_precision: 0.0004899558844044805
no_comp: 10, loss: bpr, learn_r: 0.05, item_alpha 1, user_alpha 0
train_precision: 0.00104041607119143, test_precision: 0.0015678588533774018
no_comp: 10, loss: bpr, learn_r: 0.05, item_alpha 1, user_alpha 0.1
train_precision: 0.00016006402438506484, test_precision: 0.00019598237122409046
no_comp: 10, loss: bpr, learn_r: 0.05, item_alpha 1, user_alpha 0.5
train_precision: 0.0032813125289976597, test_precision: 0.003527682740241289
no_comp: 10, loss: bpr, learn_r: 0.05, item_alpha 1, user_alpha 1
train_precision: 0.0002400960511295125, test_precision: 0.0
no_comp: 10, loss: bpr, learn_r: 0.1, item_alpha 0, user_alpha 0
train_precision: 0.001040416187606752, test_precision: 0.0015678590862080455
no_comp: 10, loss: bpr, learn_r: 0.1, item_alpha 0, user_alpha 0.1
train_precision: 0.002961184596642852, test_precision: 0.0021558059379458427
no_comp: 10, loss: bpr, learn_r: 0.1, item_alpha 0, user_alpha 0.5
train_precision: 0.0008003201219253242, test_precision: 0.0004899558844044805
no_comp: 10, loss: bpr, learn_r: 0.1, item_alpha 0, user_alpha 1
train_precision: 0.0019207684090361, test_precision: 0.0008819206850603223
no_comp: 10, loss: bpr, learn_r: 0.1, item_alpha 0.1, user_alpha 0
train_precision: 0.00104041607119143, test_precision: 0.0015678590862080455
no_comp: 10, loss: bpr, learn_r: 0.1, item_alpha 0.1, user_alpha 0.1
train_precision: 0.004641856532543898, test_precision: 0.0006859382847324014
no_comp: 10, loss: bpr, learn_r: 0.1, item_alpha 0.1, user_alpha 0.5
train_precision: 0.0032813125289976597, test_precision: 0.003527682740241289
no_comp: 10, loss: bpr, learn_r: 0.1, item_alpha 0.1, user_alpha 1
train_precision: 0.0004001600609626621, test_precision: 0.0010779029689729214
no_comp: 10, loss: bpr, learn_r: 0.1, item_alpha 0.5, user_alpha 0
train_precision: 0.00016006402438506484, test_precision: 0.000979911768808961
no_comp: 10, loss: bpr, learn_r: 0.1, item_alpha 0.5, user_alpha 0.1
train_precision: 0.00016006402438506484, test_precision: 0.002253797138109803
no_comp: 10, loss: bpr, learn_r: 0.1, item_alpha 0.5, user_alpha 0.5
train_precision: 8.003201219253242e-05, test_precision: 0.0
no_comp: 10, loss: bpr, learn_r: 0.1, item_alpha 0.5, user_alpha 1
train_precision: 0.00096038420451805, test_precision: 9.799118561204523e-05
no_comp: 10, loss: bpr, learn_r: 0.1, item_alpha 1, user_alpha 0
train_precision: 0.0003201280487701297, test_precision: 0.0015678589697927237
no_comp: 10, loss: bpr, learn_r: 0.1, item_alpha 1, user_alpha 0.1
train_precision: 8.003201219253242e-05, test_precision: 0.002253797138109803
no_comp: 10, loss: bpr, learn_r: 0.1, item_alpha 1, user_alpha 0.5
train_precision: 8.003201219253242e-05, test_precision: 0.0


no_comp: 20, loss: bpr, learn_r: 0, item_alpha 0, user_alpha 0
train_precision: 0.00096038420451805, test_precision: 0.0004899558844044805
no_comp: 20, loss: bpr, learn_r: 0, item_alpha 0, user_alpha 0.1
train_precision: 0.00096038420451805, test_precision: 0.0004899558844044805
no_comp: 20, loss: bpr, learn_r: 0, item_alpha 0, user_alpha 0.5
train_precision: 0.00096038420451805, test_precision: 0.0004899558844044805
no_comp: 20, loss: bpr, learn_r: 0, item_alpha 0, user_alpha 1
train_precision: 0.00096038420451805, test_precision: 0.0004899558844044805
no_comp: 20, loss: bpr, learn_r: 0, item_alpha 0.1, user_alpha 0
train_precision: 0.00096038420451805, test_precision: 0.0004899558844044805
no_comp: 20, loss: bpr, learn_r: 0, item_alpha 0.1, user_alpha 0.1
train_precision: 0.00096038420451805, test_precision: 0.0004899558844044805
no_comp: 20, loss: bpr, learn_r: 0, item_alpha 0.1, user_alpha 0.5
train_precision: 0.00096038420451805, test_precision: 0.0004899558844044805
no_comp: 20, loss: bpr, learn_r: 0, item_alpha 0.1, user_alpha 1
train_precision: 0.00096038420451805, test_precision: 0.0004899558844044805
no_comp: 20, loss: bpr, learn_r: 0, item_alpha 0.5, user_alpha 0
train_precision: 0.00096038420451805, test_precision: 0.0004899558844044805
no_comp: 20, loss: bpr, learn_r: 0, item_alpha 0.5, user_alpha 0.1
train_precision: 0.00096038420451805, test_precision: 0.0004899558844044805
no_comp: 20, loss: bpr, learn_r: 0, item_alpha 0.5, user_alpha 0.5
train_precision: 0.00096038420451805, test_precision: 0.0004899558844044805
no_comp: 20, loss: bpr, learn_r: 0, item_alpha 0.5, user_alpha 1
train_precision: 0.00096038420451805, test_precision: 0.0004899558844044805
no_comp: 20, loss: bpr, learn_r: 0, item_alpha 1, user_alpha 0
train_precision: 0.00096038420451805, test_precision: 0.0004899558844044805
no_comp: 20, loss: bpr, learn_r: 0, item_alpha 1, user_alpha 0.1
train_precision: 0.00096038420451805, test_precision: 0.0004899558844044805
no_comp: 20, loss: bpr, learn_r: 0, item_alpha 1, user_alpha 0.5
train_precision: 0.00096038420451805, test_precision: 0.0004899558844044805
no_comp: 20, loss: bpr, learn_r: 0, item_alpha 1, user_alpha 1
train_precision: 0.00096038420451805, test_precision: 0.0004899558844044805
no_comp: 20, loss: bpr, learn_r: 0.05, item_alpha 0, user_alpha 0
train_precision: 0.0013605442363768816, test_precision: 0.0011758941691368818
no_comp: 20, loss: bpr, learn_r: 0.05, item_alpha 0, user_alpha 0.1
train_precision: 0.00104041607119143, test_precision: 0.003821656107902527
no_comp: 20, loss: bpr, learn_r: 0.05, item_alpha 0, user_alpha 0.5
train_precision: 0.007843137718737125, test_precision: 9.799118561204523e-05
no_comp: 20, loss: bpr, learn_r: 0.05, item_alpha 0, user_alpha 1
train_precision: 0.0005602241144515574, test_precision: 0.0013718765694648027
no_comp: 20, loss: bpr, learn_r: 0.05, item_alpha 0.1, user_alpha 0
train_precision: 0.0006402560975402594, test_precision: 0.0023517883382737637
no_comp: 20, loss: bpr, learn_r: 0.05, item_alpha 0.1, user_alpha 0.1
train_precision: 0.0, test_precision: 0.0011758941691368818
no_comp: 20, loss: bpr, learn_r: 0.05, item_alpha 0.1, user_alpha 0.5
train_precision: 0.0, test_precision: 0.0015678589697927237
no_comp: 20, loss: bpr, learn_r: 0.05, item_alpha 0.1, user_alpha 1
train_precision: 0.0, test_precision: 0.0006859382847324014
no_comp: 20, loss: bpr, learn_r: 0.05, item_alpha 0.5, user_alpha 0
train_precision: 0.0003201280487701297, test_precision: 0.0016658501699566841
no_comp: 20, loss: bpr, learn_r: 0.05, item_alpha 0.5, user_alpha 0.1
train_precision: 0.00016006402438506484, test_precision: 0.0007839294848963618
no_comp: 20, loss: bpr, learn_r: 0.05, item_alpha 0.5, user_alpha 0.5
train_precision: 0.002641056664288044, test_precision: 0.0012738852528855205
no_comp: 20, loss: bpr, learn_r: 0.05, item_alpha 0.5, user_alpha 1
train_precision: 0.0002400960511295125, test_precision: 0.0008819206268526614
no_comp: 20, loss: bpr, learn_r: 0.05, item_alpha 1, user_alpha 0
train_precision: 0.002160864183679223, test_precision: 0.0011758941691368818
no_comp: 20, loss: bpr, learn_r: 0.05, item_alpha 1, user_alpha 0.1
train_precision: 0.00096038420451805, test_precision: 0.0016658501699566841
no_comp: 20, loss: bpr, learn_r: 0.05, item_alpha 1, user_alpha 0.5
train_precision: 0.018727490678429604, test_precision: 0.000979911768808961
no_comp: 20, loss: bpr, learn_r: 0.05, item_alpha 1, user_alpha 1
train_precision: 0.002240896224975586, test_precision: 0.0013718765694648027
no_comp: 20, loss: bpr, learn_r: 0.1, item_alpha 0, user_alpha 0
train_precision: 0.00096038420451805, test_precision: 0.0017638413701206446
no_comp: 20, loss: bpr, learn_r: 0.1, item_alpha 0, user_alpha 0.1
train_precision: 0.0022408964578062296, test_precision: 0.0010779029689729214
no_comp: 20, loss: bpr, learn_r: 0.1, item_alpha 0, user_alpha 0.5
train_precision: 0.006162464618682861, test_precision: 0.0017638413701206446
no_comp: 20, loss: bpr, learn_r: 0.1, item_alpha 0, user_alpha 1
train_precision: 0.0040816329419612885, test_precision: 0.0008819206850603223
no_comp: 20, loss: bpr, learn_r: 0.1, item_alpha 0.1, user_alpha 0
train_precision: 8.003201219253242e-05, test_precision: 0.0011758941691368818
no_comp: 20, loss: bpr, learn_r: 0.1, item_alpha 0.1, user_alpha 0.1
train_precision: 0.0011204482289031148, test_precision: 0.0018618324538692832
no_comp: 20, loss: bpr, learn_r: 0.1, item_alpha 0.1, user_alpha 0.5
train_precision: 0.03825530409812927, test_precision: 0.0006859382847324014
no_comp: 20, loss: bpr, learn_r: 0.1, item_alpha 0.1, user_alpha 1
train_precision: 0.00016006402438506484, test_precision: 0.0006859382847324014
no_comp: 20, loss: bpr, learn_r: 0.1, item_alpha 0.5, user_alpha 0
train_precision: 0.0013605442363768816, test_precision: 0.0011758941691368818
no_comp: 20, loss: bpr, learn_r: 0.1, item_alpha 0.5, user_alpha 0.1
train_precision: 0.0004001600609626621, test_precision: 0.0008819206268526614
no_comp: 20, loss: bpr, learn_r: 0.1, item_alpha 0.5, user_alpha 0.5
train_precision: 0.018007202073931694, test_precision: 0.0012738852528855205
no_comp: 20, loss: bpr, learn_r: 0.1, item_alpha 0.5, user_alpha 1
train_precision: 0.0022408964578062296, test_precision: 0.0013718765694648027

no_comp: 30, loss: bpr, learn_r: 0.05, item_alpha 0, user_alpha 0
train_precision: 0.0012004801537841558, test_precision: 0.0010779029689729214
no_comp: 30, loss: bpr, learn_r: 0.05, item_alpha 0, user_alpha 0.1
train_precision: 0.0012004801537841558, test_precision: 0.0016658501699566841
no_comp: 30, loss: bpr, learn_r: 0.05, item_alpha 0, user_alpha 0.5
train_precision: 0.002561024622991681, test_precision: 0.0010779029689729214
no_comp: 30, loss: bpr, learn_r: 0.05, item_alpha 0, user_alpha 0.9
train_precision: 0.0024009605403989553, test_precision: 0.0021558059379458427
no_comp: 30, loss: bpr, learn_r: 0.05, item_alpha 0.1, user_alpha 0
train_precision: 0.00016006402438506484, test_precision: 0.0006859382265247405
no_comp: 30, loss: bpr, learn_r: 0.05, item_alpha 0.1, user_alpha 0.1
train_precision: 0.0, test_precision: 0.0011758941691368818
no_comp: 30, loss: bpr, learn_r: 0.05, item_alpha 0.1, user_alpha 0.5
train_precision: 0.0, test_precision: 0.0006859382847324014
no_comp: 30, loss: bpr, learn_r: 0.05, item_alpha 0.1, user_alpha 0.9
train_precision: 0.0, test_precision: 0.0
no_comp: 30, loss: bpr, learn_r: 0.05, item_alpha 0.5, user_alpha 0
train_precision: 0.0004001600609626621, test_precision: 0.0017638412537053227
no_comp: 30, loss: bpr, learn_r: 0.05, item_alpha 0.5, user_alpha 0.1
train_precision: 0.0035214086528867483, test_precision: 0.0015678589697927237
no_comp: 30, loss: bpr, learn_r: 0.05, item_alpha 0.5, user_alpha 0.5
train_precision: 0.0035214086528867483, test_precision: 0.0010779029689729214
no_comp: 30, loss: bpr, learn_r: 0.05, item_alpha 0.5, user_alpha 0.9
train_precision: 0.050340138375759125, test_precision: 0.0021558059379458427
no_comp: 30, loss: bpr, learn_r: 0.05, item_alpha 0.9, user_alpha 0
train_precision: 0.0011204482289031148, test_precision: 0.0010779029689729214
no_comp: 30, loss: bpr, learn_r: 0.05, item_alpha 0.9, user_alpha 0.1
train_precision: 0.0035214086528867483, test_precision: 0.0010779029689729214
no_comp: 30, loss: bpr, learn_r: 0.05, item_alpha 0.9, user_alpha 0.5
train_precision: 0.004241697024554014, test_precision: 0.0008819206850603223
no_comp: 30, loss: bpr, learn_r: 0.05, item_alpha 0.9, user_alpha 0.9
train_precision: 0.0022408964578062296, test_precision: 0.0008819206850603223
no_comp: 30, loss: bpr, learn_r: 0.5, item_alpha 0, user_alpha 0
train_precision: 0.010324129834771156, test_precision: 0.002645761938765645
no_comp: 30, loss: bpr, learn_r: 0.5, item_alpha 0, user_alpha 0.1
train_precision: 0.009683873504400253, test_precision: 0.0007839294848963618
no_comp: 30, loss: bpr, learn_r: 0.5, item_alpha 0, user_alpha 0.5
train_precision: 0.0021608644165098667, test_precision: 0.0010779029689729214
no_comp: 30, loss: bpr, learn_r: 0.5, item_alpha 0, user_alpha 0.9
train_precision: 8.003201219253242e-05, test_precision: 0.000979911768808961
no_comp: 30, loss: bpr, learn_r: 0.5, item_alpha 0.1, user_alpha 0
train_precision: 0.0016006402438506484, test_precision: 0.0008819206268526614
no_comp: 30, loss: bpr, learn_r: 0.5, item_alpha 0.1, user_alpha 0.1
train_precision: 0.005282113328576088, test_precision: 0.0008819206850603223
no_comp: 30, loss: bpr, learn_r: 0.5, item_alpha 0.1, user_alpha 0.5
train_precision: 0.00016006402438506484, test_precision: 0.0010779029689729214
no_comp: 30, loss: bpr, learn_r: 0.5, item_alpha 0.1, user_alpha 0.9
train_precision: 0.03257302939891815, test_precision: 0.00029397354228422046